In [1]:
import os
import pm4py
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.algo.evaluation.replay_fitness import algorithm as fitness_alg
from pm4py.algo.evaluation.precision import algorithm as precision_alg
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_alg
from pm4py.algo.evaluation.generalization import algorithm as generalization_alg

In [ ]:
# Iterate over all Event Logs used for evaluation in the DPIM paper

directory: str = "./Evaluation Logs"
for file in os.listdir(directory):
    if file.endswith(".gz"): 
        filename: str = os.path.join(directory, file)
        print(f"============== PROCESSING {file} ==============")
        
        # 1) Load log (.xes.gz)
        log = pm4py.read_xes(filename)
        
        # 2) Baseline IM using pm4py (we compare the DPIM tree against this one)
        im_tree = pm4py.discover_process_tree_inductive(log)
        
        # 3) Convert tree -> Petri net
        net, im, fm = pt_converter.apply(im_tree)
        
        # 4) Metrics
        fitness = fitness_alg.apply(log, net, im, fm)                 # dict, contains 'averageFitness' or similar
        precision = precision_alg.apply(log, net, im, fm)             # float
        simplicity = simplicity_alg.apply(net)                        # float (model-only)
        generalization = generalization_alg.apply(log, net, im, fm)   # float

        print("fitness:", round(number=fitness['log_fitness'], ndigits=4))
        print("precision:", round(number=precision, ndigits=4))
        print("simplicity:", round(number=simplicity, ndigits=4))
        print("generalization:", round(number=generalization, ndigits=4))
        print(f"============== END OF {file} ==============")
        input("Press Enter to continue...")

============== PROCESSING BPIC 2015_1.xes.gz ==============


/home/petrov/Desktop/BI2/Group Project/Business-Intelligence-WS2025/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1199/1199 [00:02<00:00, 542.30it/s]
